In [78]:
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)
#source('tests.R')
#source("cleanup.R")
set.seed(7493)

Here we are reading the data.

In [79]:
match_data <- read_csv("data/season_2018.csv") %>% 
            mutate(FTR = as_factor(FTR)) %>% 
            mutate(FTHG = as.integer(FTHG)) %>% 
            mutate(FTAG = as.integer(FTAG)) %>% 
            mutate(HomeTeam = as_factor(HomeTeam)) %>% 
            mutate(AwayTeam = as_factor(AwayTeam))
match_data 

Parsed with column specification:
cols(
  .default = col_double(),
  Div = col_character(),
  Date = col_character(),
  HomeTeam = col_character(),
  AwayTeam = col_character(),
  FTR = col_character(),
  HTR = col_character(),
  Referee = col_character()
)

See spec(...) for full column specifications.



Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,⋯,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
<chr>,<chr>,<fct>,<fct>,<int>,<int>,<fct>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
E0,10/08/2018,Man United,Leicester,2,1,H,1,0,H,⋯,1.79,17,-0.75,1.75,1.70,2.29,2.21,1.55,4.07,7.69
E0,11/08/2018,Bournemouth,Cardiff,2,0,H,1,0,H,⋯,1.83,20,-0.75,2.20,2.13,1.80,1.75,1.88,3.61,4.70
E0,11/08/2018,Fulham,Crystal Palace,0,2,A,0,1,A,⋯,1.87,22,-0.25,2.18,2.11,1.81,1.77,2.62,3.38,2.90
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
E0,12/05/2019,Southampton,Huddersfield,1,1,D,1,0,H,⋯,2.29,22,-1.5,2.27,2.16,1.80,1.73,1.37,5.36,8.49
E0,12/05/2019,Tottenham,Everton,2,2,D,1,0,H,⋯,2.07,19,-0.5,2.13,2.08,1.85,1.80,1.91,3.81,4.15
E0,12/05/2019,Watford,West Ham,1,4,A,0,2,A,⋯,2.44,19,-0.5,2.25,2.19,1.78,1.72,2.11,3.86,3.41


In [80]:
match_split <- initial_split(match_data, prop = 3/4, strata = "FTR")  
match_train <- training(match_split)   
match_test <- testing(match_split)


Below represents the number of matches won by each home team. 

In [81]:
training_summary_netscore <- match_train %>% 
                group_by(FTR, HomeTeam, AwayTeam, FTHG, FTAG) %>%
                summarize() %>% 
                mutate(Net_Score = FTHG - FTAG)

training_summary_nwin_home <- match_train %>% 
                group_by(FTR, HomeTeam) %>%
                summarize(TotalWinsHome = n()) %>% 
                arrange(desc(TotalWinsHome))

training_summary_nwin_away <- match_train %>% 
                group_by(FTR, AwayTeam) %>%
                summarize(TotalWinsAway = n()) %>% 
                arrange(desc(TotalWinsAway))


training_summary_nwin_home
training_summary_nwin_away
training_summary_netscore

`summarise()` regrouping output by 'FTR', 'HomeTeam', 'AwayTeam', 'FTHG' (override with `.groups` argument)

`summarise()` regrouping output by 'FTR' (override with `.groups` argument)

`summarise()` regrouping output by 'FTR' (override with `.groups` argument)



FTR,HomeTeam,TotalWinsHome
<fct>,<fct>,<int>
H,Liverpool,15
H,Man City,15
H,Arsenal,12
⋮,⋮,⋮
D,Newcastle,1
D,Cardiff,1
D,Everton,1


FTR,AwayTeam,TotalWinsAway
<fct>,<fct>,<int>
H,Fulham,12
H,Huddersfield,12
A,Man City,11
⋮,⋮,⋮
D,Chelsea,1
D,Man City,1
D,Bournemouth,1


FTR,HomeTeam,AwayTeam,FTHG,FTAG,Net_Score
<fct>,<fct>,<fct>,<int>,<int>,<int>
H,Man United,Brighton,2,1,1
H,Man United,Everton,2,1,1
H,Man United,West Ham,2,1,1
⋮,⋮,⋮,⋮,⋮,⋮
D,Crystal Palace,Everton,0,0,0
D,Crystal Palace,West Ham,1,1,0
D,Crystal Palace,Newcastle,0,0,0
